# [Report] Word-level Spelling Correction for Burmese Language using Statistical Approaches

References:
- [1] Vacláv Chvátal and David Sankoff. "Longest common subsequences of two random sequences", 1975. Journal of Applied Probability, Python module: ngram (https://pypi.org/project/ngram/).

- [2] Wolf Garbe <wolf.garbe@faroo.com> Description: https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f .URL: https://github.com/wolfgarbe/symspell .Python module: symspellpy (https://github.com/mammothb/symspellpy)

- [3] Mon, Ei & Kyaw Thu, Ye & Yu, Than & Oo, Aye. (2021). SymSpell4Burmese: Symmetric Delete Spelling Correction Algorithm (SymSpell) for Burmese Spelling Checking. 1-6. 10.1109/iSAI-NLP54397.2021.9678171. 

- [4] Zar Zar Hlaing, Ye Kyaw Thu, Myat Myo Nwe Wai, Thepchai Supnithi, Ponrudee Netisopakul, "Myanmar POS resource extension effects on automatic tagging methods", In Proceedings of the 15th International Joint Symposium on Artificial Intelligence and Natural Language Processing (iSAI-NLP 2020), Nov 18 to Nov 20, 2020, Bangkok, Thailand, pp. 189-194.*

- [5] myWord: Syllable, Word and Phrase Segmenter for Burmese, Ye Kyaw Thu, Sept 2021, GitHub Link: https://github.com/ye-kyaw-thu/myWord**

- [6] https://bhashkarkunal.medium.com/spelling-correction-using-deep-learning-how-bi-directional-lstm-with-attention-flow-works-in-366fabcc7a2f

\* I used myPOS ver3 (without POS-tags) for building dictionaries

\** built dictionaries using myWord tool

In [1]:
!pip install symspellpy ngram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 8.7 MB/s 
     |████████████████████████████████| 125 kB 55.2 MB/s 


In [ ]:
import os
import re
import string

from ngram import NGram
from symspellpy.symspellpy import SymSpell
from symspellpy.editdistance import DistanceAlgorithm

In [ ]:
corpus_path = "/content/myPOS.word"
bigram_path = "/content/small_frequency_bigram_dictionary_mm"
dictionary_path = "/content/small_frequency__dictionary_mm"
corpus = " ".join(open(corpus_path).read().splitlines())

In [ ]:
def ngram_similarity(sentences,n):
  ngram = NGram(corpus.split(), N=n)
  predicts = []
  if not isinstance(sentences, list):
            sentences = [sentences]
            
  for i in range(len(sentences)):
            split = []

            for x in sentences[i].split():
                sugg = ngram.find(x) if x not in string.punctuation else None
                split.append(sugg if sugg else x)

            predicts.append(" ".join(split))
  return predicts

In [ ]:
def mySymSpell(sentences,distance):
  symspell = SymSpell(max_dictionary_edit_distance=distance)
  symspell._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  
  symspell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
  
  symspell.load_dictionary(dictionary_path, term_index=0, count_index=1)
  
  predicts = []

  # Word Segmentation using SymSpell
  #sentences = symspell.word_segmentation(sentences, 0)[0]

  if not isinstance(sentences, list):
        sentences = [sentences]

  for i in range(len(sentences)):
            split = []

            for x in sentences[i].split():
                sugg = symspell.lookup(x, verbosity=0, max_edit_distance=distance,
                                       transfer_casing=True) if x not in string.punctuation else None
                split.append(sugg[0].term if sugg else x)

            predicts.append(" ".join(split))

  return predicts

In [ ]:
#phonetic error
sent = "ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းခလေး တွေ ရဲ့ ပုံစံ"

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
Edit distance 2 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
Edit distance 3 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
Edit distance 4 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
Edit distance 5 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
Edit distance 6 ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']


In [ ]:
#with symspell word segmentaion
#phonetic error

sent = "ဖောက်ပြန်ဖို့သေချာသလောက်ရှိနေတဲ့မိန်းခလေးတွေရဲ့ပုံစံ"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', corrected_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', distance_sum=12, log_prob_sum=-119.73672313520763)
Edit distance 2
Composition(segmented_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', corrected_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', distance_sum=12, log_prob_sum=-119.73672313520763)
Edit distance 3
Composition(segmented_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', corrected_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', distance_sum=12, log_prob_sum=-119.73672313520763)
Edit distance 4
Composition(segmented_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', corrected_string='ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံစံ', distance_sum=12, log_prob_sum=-119.73672313520763)
Edit

In [ ]:
sent = "ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းခလေး တွေ ရဲ့ ပုံစံ"

for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
2-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
3-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
4-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
5-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']
6-gram  ['ဖောက်ပြန် ဖို့ သေချာ သလောက် ရှိ နေ တဲ့ မိန်းကလေး တွေ ရဲ့ ပုံစံ']


In [ ]:
#typographic error
sent = "အမေ ဒေါ်အောင်ဆန်းစုကြည် တဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထပ် ပန်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။ "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပန်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
Edit distance 2 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
Edit distance 3 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
Edit distance 4 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
Edit distance 5 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
Edit distance 6 ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုဖာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး 

In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပြန်ဆင်း ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
2-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပန် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
3-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပန် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
4-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပန် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
5-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']
6-gram  ['အမေ ဒေါ်အောင်ဆန်းစုကြည် ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆိုတာ ဦးထုပ် ပြင်ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။']


In [ ]:
#with symspell word segmentaion
#typographic error

sent = "အမေဒေါ်အောင်ဆန်းစုကြည်တဖွဖွပြောခဲ့တဲ့စကားအတိုင်းပြည်သူသာအဓိကဖြစ်တယ်ဆိုတာဦးထပ်ပန်ဆင်ကြရင်းသွေးမအေးကြဖို့တိုက်တွန်းလိုက်ရပါတယ်။"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='အမေ ဒေါ်အောင်ဆန်းစုကြည် တ ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မအေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။', corrected_string='အမေ ဒေါ်အောင်ဆန်းစုကြည် တ ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မအေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။', distance_sum=30, log_prob_sum=-290.4834926650544)
Edit distance 2
Composition(segmented_string='အမေ ဒေါ်အောင်ဆန်းစုကြည် တ ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မအေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။', corrected_string='အမေ ဒေါ်အောင်ဆန်းစုကြည် တ ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မအေး ကြ ဖို့ တိုက်တွန်း လိုက် ရ ပါတယ် ။', distance_sum=30, log_prob_sum=-290.4834926650544)
Edit distance 3
Composition(segmented_string='အမေ ဒေါ်အောင်ဆန်းစုကြည် တ ဖွဖွ ပြော ခဲ့ တဲ့ စကား အတိုင်း ပြည်သူ သာ အဓိက ဖြစ် တယ် ဆို တာ ဦး ထပ်

In [ ]:
#slang error, pho+typo error, typo error

sent = "ဟု တရ် သစ္စာဖောက် ခြင်း ဆိုတာ ငါတို့ ရန်သူ တေ ဆီက ဘယ်တော့မှ လာ တာ မ ဟုတ် ဘူးးး "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ ငါတို့ ရန်သူ တ ဆက ဘယ်တော့မှ လာ တာ မ ဟုတ် ဘူးးး']
Edit distance 2 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ တို့ ရန်သူ တ ဆက ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
Edit distance 3 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ တို့ ရန်သူ တ ဆက ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
Edit distance 4 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ တို့ ရန်သူ တ ဆက ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
Edit distance 5 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ တို့ ရန်သူ တ ဆက ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
Edit distance 6 ['ဟု တရ သစ္စာဖောက် ခြင်း ဆိုဖာ တို့ ရန်သူ တ ဆက ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']


In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ဟု ရိတ် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ကဆီ ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
2-gram  ['ဟု တရက် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ဆီ ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
3-gram  ['ဟု တရက် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ဆီ ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
4-gram  ['ဟု တရက် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ဆီကြိတ် ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
5-gram  ['ဟု တရက် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ဆီကြိတ် ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']
6-gram  ['ဟု တရက် သစ္စာဖောက် ခြင်း ဆိုတာ တို့ ရန်သူ တေ ဆီကြိတ် ဘယ်တော့ လာ တာ မ ဟုတ် ဘူး']


In [ ]:
#with symspell word segmentaion
#slang error, pho+typo error, typo error

sent = "ဟုတရ်သစ္စာဖောက်ခြင်းဆိုတာငါတို့ရန်သူတေဆီကဘယ်တော့မှလာတာမဟုတ်ဘူးးး"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', corrected_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', distance_sum=24, log_prob_sum=-191.363340270537)
Edit distance 2
Composition(segmented_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', corrected_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', distance_sum=24, log_prob_sum=-191.363340270537)
Edit distance 3
Composition(segmented_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', corrected_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', distance_sum=24, log_prob_sum=-191.363340270537)
Edit distance 4
Composition(segmented_string='ဟု တရ ် သစ္စာဖောက် ခြင်း ဆို တာ ငါ တို့ ရန်သူ တေ ဆီ က ဘယ်တော့ မှ လာ တာ မဟုတ် ဘူး းး', corrected_str

In [ ]:
# Consonant error, sequence error

sent = "ကြက်ဉ ၁ လုံး ၁၅၀၀ ြပော ရျ် ရန်ဖြစ်"

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']
Edit distance 2 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']
Edit distance 3 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']
Edit distance 4 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']
Edit distance 5 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']
Edit distance 6 ['ကြက်ဥ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန်ဖြစ်']


In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ကြက် ၁ လုံး ၁၅၀၀ ပြော ရချင် ရန်ဖြစ်']
2-gram  ['ကြက် ၁ လုံး ၁၅၀၀ ပေါလော ရင် ရန်ဖြစ်']
3-gram  ['ကြက် ၁ လုံး ၁၅၀၀ သော ရင် ရန်ဖြစ်']
4-gram  ['ကြက် ၁ လုံး ၁၅၀၀ သော ရင် ရန်ဖြစ်']
5-gram  ['ကြက် ၁ လုံး ၁၅၀၀ သော ရင် ရန်ဖြစ်']
6-gram  ['ကြက် ၁ လုံး ၁၅၀၀ သော ရင် ရန်ဖြစ်']


In [ ]:
#with symspell word segmentaion
#Consonant error, sequence error

sent = "ကြက်ဉ၁လုံး၁၅၀၀ ြပောရျ်ရန်ဖြစ်"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', corrected_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', distance_sum=12, log_prob_sum=-73.94305229935105)
Edit distance 2
Composition(segmented_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', corrected_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', distance_sum=12, log_prob_sum=-73.94305229935105)
Edit distance 3
Composition(segmented_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', corrected_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', distance_sum=12, log_prob_sum=-73.94305229935105)
Edit distance 4
Composition(segmented_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', corrected_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', distance_sum=12, log_prob_sum=-73.94305229935105)
Edit distance 5
Composition(segmented_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', corrected_string='ကြက် ဉ ၁ လုံး ၁၅၀၀ ြပောရျ် ရန်ဖြစ်', distance_sum=12, log_prob_sum=-73.94305229935105)
Edit distance 6
Composition(segmented_string='ကြက် ဉ ၁ လုံး 

In [ ]:
# dialect error
sent = "စပ်စု လတ် သာ အေ "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['စပ်စု လတ် သာ အေ']
Edit distance 2 ['အုပ်စု လတ် သာ အေ']
Edit distance 3 ['အုပ်စု လတ် သာ အေ']
Edit distance 4 ['အုပ်စု လတ် သာ အေ']
Edit distance 5 ['အုပ်စု လတ် သာ အေ']
Edit distance 6 ['အုပ်စု လတ် သာ အေ']


In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['စပ်စု လတ် သာ အေ']
2-gram  ['စပ်စု လတ် သာ အေ']
3-gram  ['စပ်စု လတ် သာ အေ']
4-gram  ['စပ်စု လတ် သာ အေ']
5-gram  ['စပ်စု လတ် သာ အေ']
6-gram  ['စပ်စု လတ် သာ အေ']


In [ ]:
#with symspell word segmentaion
# dialect error

sent = "စပ်စုလတ်သာအေ "

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)
Edit distance 2
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)
Edit distance 3
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)
Edit distance 4
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)
Edit distance 5
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)
Edit distance 6
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-62.87497840977267)


In [ ]:
# Common spelling mistakes Consonant errors
common1 = "ည က အိမ်မက် က ကွာ ၊ "
common2 = "မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ ာ ပါ လိုက်ရှာ နေ ရော"

In [ ]:
for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(common1,i))

Edit distance 1 ['ည က အိမ်မက် က ကွာ ၊']
Edit distance 2 ['ည က အိမ်မက် က ကွာ ၊']
Edit distance 3 ['ည က အိမ်မက် က ကွာ ၊']
Edit distance 4 ['ည က အိမ်မက် က ကွာ ၊']
Edit distance 5 ['ည က အိမ်မက် က ကွာ ၊']
Edit distance 6 ['ည က အိမ်မက် က ကွာ ၊']


In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(common1,i))

1-gram  ['ည က အိမ်မက် က ကွာ ၊']
2-gram  ['ည က အိမ်မက် က ကွာ ၊']
3-gram  ['ည က အိမ်မက် က ကွာ ၊']
4-gram  ['ည က အိမ်မက် က ကွာ ၊']
5-gram  ['ည က အိမ်မက် က ကွာ ၊']
6-gram  ['ည က အိမ်မက် က ကွာ ၊']


In [ ]:
for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(common2,i))

Edit distance 1 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']
Edit distance 2 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']
Edit distance 3 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']
Edit distance 4 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']
Edit distance 5 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']
Edit distance 6 ['မက် ရင်း ကန ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ က ပါ လိုက်ရှာ နေ ရော']


In [ ]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(common2,i))

1-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ ပါ လိုက်ရှာ နေ ရော']
2-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ ပါ လိုက်ရှာ နေ ရော']
3-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ မျောပါ လိုက်ရှာ နေ ရော']
4-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ မျောပါ လိုက်ရှာ နေ ရော']
5-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ မျောပါ လိုက်ရှာ နေ ရော']
6-gram  ['မက် ရင်း ကနေ ရောင်ရမ်း ပြီး ပျောက်သွား တဲ့ ပစ္စည်း ကို အပြင် မှ မျောပါ လိုက်ရှာ နေ ရော']


# [Report] Syllable-level Spelling Correction for Burmese Language using Statistical Approaches

Author : Thura Aung

References:
- [1] Vacláv Chvátal and David Sankoff. "Longest common subsequences of two random sequences", 1975. Journal of Applied Probability, Python module: ngram (https://pypi.org/project/ngram/).

- [2] Wolf Garbe <wolf.garbe@faroo.com> Description: https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f .URL: https://github.com/wolfgarbe/symspell .Python module: symspellpy (https://github.com/mammothb/symspellpy)

- [3] Mon, Ei & Kyaw Thu, Ye & Yu, Than & Oo, Aye. (2021). SymSpell4Burmese: Symmetric Delete Spelling Correction Algorithm (SymSpell) for Burmese Spelling Checking. 1-6. 10.1109/iSAI-NLP54397.2021.9678171. 

- [4] Zar Zar Hlaing, Ye Kyaw Thu, Myat Myo Nwe Wai, Thepchai Supnithi, Ponrudee Netisopakul, "Myanmar POS resource extension effects on automatic tagging methods", In Proceedings of the 15th International Joint Symposium on Artificial Intelligence and Natural Language Processing (iSAI-NLP 2020), Nov 18 to Nov 20, 2020, Bangkok, Thailand, pp. 189-194.*

- [5] myWord: Syllable, Word and Phrase Segmenter for Burmese, Ye Kyaw Thu, Sept 2021, GitHub Link: https://github.com/ye-kyaw-thu/myWord**

- [6] https://bhashkarkunal.medium.com/spelling-correction-using-deep-learning-how-bi-directional-lstm-with-attention-flow-works-in-366fabcc7a2f

\* I used myPOS ver3 (without POS-tags) for building dictionaries

\** built dictionaries using myWord tool

In [2]:
!pip install symspellpy ngram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import re
import string

from ngram import NGram
from symspellpy.symspellpy import SymSpell
from symspellpy.editdistance import DistanceAlgorithm

In [4]:
corpus_path = "/content/myPOS.syllable"
bigram_path = "/content/small_frequency_bigram_dictionary_mm.syl"
dictionary_path = "/content/small_frequency_dictionary_mm.syl"
corpus = " ".join(open(corpus_path).read().splitlines())

In [5]:
def ngram_similarity(sentences,n):
  ngram = NGram(corpus.split(), N=n)
  predicts = []
  if not isinstance(sentences, list):
            sentences = [sentences]
            
  for i in range(len(sentences)):
            split = []

            for x in sentences[i].split():
                sugg = ngram.find(x) if x not in string.punctuation else None
                split.append(sugg if sugg else x)

            predicts.append(" ".join(split))
  return predicts

In [6]:
def mySymSpell(sentences,distance):
  symspell = SymSpell(max_dictionary_edit_distance=distance)
  symspell._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  
  symspell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)
  
  symspell.load_dictionary(dictionary_path, term_index=0, count_index=1)
  
  predicts = []

  # Word Segmentation using SymSpell
  #sentences = symspell.word_segmentation(sentences, 0)[0]

  if not isinstance(sentences, list):
        sentences = [sentences]

  for i in range(len(sentences)):
            split = []

            for x in sentences[i].split():
                sugg = symspell.lookup(x, verbosity=0, max_edit_distance=distance,
                                       transfer_casing=True) if x not in string.punctuation else None
                split.append(sugg[0].term if sugg else x)

            predicts.append(" ".join(split))

  return predicts

In [8]:
#phonetic error
sent = "ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ"

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
Edit distance 2 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
Edit distance 3 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
Edit distance 4 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
Edit distance 5 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
Edit distance 6 ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']


In [9]:
#with symspell word segmentaion
#phonetic error

sent = "ဖောက်ပြန်ဖို့သေချာသလောက်ရှိနေတဲ့မိန်းခလေးတွေရဲ့ပုံစံ"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', corrected_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', distance_sum=16, log_prob_sum=-151.1624795414868)
Edit distance 2
Composition(segmented_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', corrected_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', distance_sum=16, log_prob_sum=-151.1624795414868)
Edit distance 3
Composition(segmented_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', corrected_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', distance_sum=16, log_prob_sum=-151.1624795414868)
Edit distance 4
Composition(segmented_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', corrected_string='ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ', distance_sum=16, log_prob_

In [11]:
sent = "ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ"

for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
2-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
3-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
4-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
5-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']
6-gram  ['ဖောက် ပြန် ဖို့ သေ ချာ သ လောက် ရှိ နေ တဲ့ မိန်း ခ လေး တွေ ရဲ့ ပုံ စံ']


In [12]:
#typographic error
sent = "အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။ "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
Edit distance 2 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
Edit distance 3 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
Edit distance 4 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
Edit distance 5 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
Edit distance 6 ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖ

In [13]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
2-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
3-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
4-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
5-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။']
6-gram  ['အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ

In [14]:
#with symspell word segmentaion
#typographic error

sent = "အမေဒေါ်အောင်ဆန်းစုကြည်တဖွဖွပြောခဲ့တဲ့စကားအတိုင်းပြည်သူသာအဓိကဖြစ်တယ်ဆိုတာဦးထပ်ပန်ဆင်ကြရင်းသွေးမအေးကြဖို့တိုက်တွန်းလိုက်ရပါတယ်။"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။', corrected_string='အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။', distance_sum=44, log_prob_sum=-392.1211581828058)
Edit distance 2
Composition(segmented_string='အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။', corrected_string='အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပြော ခဲ့ တဲ့ စ ကား အ တိုင်း ပြည် သူ သာ အ ဓိ က ဖြစ် တယ် ဆို တာ ဦး ထပ် ပန် ဆင် ကြ ရင်း သွေး မ အေး ကြ ဖို့ တိုက် တွန်း လိုက် ရ ပါ တယ် ။', distance_sum=44, log_prob_sum=-392.1211581828058)
Edit distance 3
Composition(segmented_string='အ မေ ဒေါ် အောင် ဆန်း စု ကြည် တ ဖွ ဖွ ပ

In [15]:
#slang error, pho+typo error, typo error

sent = "ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူးးး "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူးးး']
Edit distance 2 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
Edit distance 3 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
Edit distance 4 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
Edit distance 5 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
Edit distance 6 ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']


In [16]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ဟု ရတ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
2-gram  ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
3-gram  ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
4-gram  ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
5-gram  ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']
6-gram  ['ဟု တယ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး']


In [17]:
#with symspell word segmentaion
#slang error, pho+typo error, typo error

sent = "ဟုတရ်သစ္စာဖောက်ခြင်းဆိုတာငါတို့ရန်သူတေဆီကဘယ်တော့မှလာတာမဟုတ်ဘူးးး"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', corrected_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', distance_sum=27, log_prob_sum=-212.2135372256042)
Edit distance 2
Composition(segmented_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', corrected_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', distance_sum=27, log_prob_sum=-212.2135372256042)
Edit distance 3
Composition(segmented_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', corrected_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ် ဘူး းး', distance_sum=27, log_prob_sum=-212.2135372256042)
Edit distance 4
Composition(segmented_string='ဟု တရ် သစ္စာ ဖောက် ခြင်း ဆို တာ ငါ တို့ ရန် သူ တေ ဆီ က ဘယ် တော့ မှ လာ တာ မ ဟုတ

In [20]:
# Consonant error, sequence error

sent = "ကြက် ဉ ၁ လုံး ၁၅၀၀  ြပော ရျ် ရန် ဖြစ်"

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['ကြက် ။ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန် ဖြစ်']
Edit distance 2 ['ကြက် ။ ၁ လုံး ၁၅၀၀ ပြော ရင် ရန် ဖြစ်']
Edit distance 3 ['ကြက် ။ ၁ လုံး ၁ ပြော ရင် ရန် ဖြစ်']
Edit distance 4 ['ကြက် ။ ၁ လုံး ၁ ပြော ရင် ရန် ဖြစ်']
Edit distance 5 ['ကြက် ။ ၁ လုံး ၁ ပြော ရင် ရန် ဖြစ်']
Edit distance 6 ['ကြက် ။ ၁ လုံး ၁ ပြော ရင် ရန် ဖြစ်']


In [21]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['ကြက် ဉာ ၁ လုံး ၁ ပြော ရင် ရန် ဖြစ်']
2-gram  ['ကြက် ဉာ ၁ လုံး ၁ ကော ရင် ရန် ဖြစ်']
3-gram  ['ကြက် ဉာ ၁ လုံး ၁ ကော ရင် ရန် ဖြစ်']
4-gram  ['ကြက် ဉာ ၁ လုံး ၁ ကော ရင် ရန် ဖြစ်']
5-gram  ['ကြက် ဉာ ၁ လုံး ၁ ကော ရင် ရန် ဖြစ်']
6-gram  ['ကြက် ဉာ ၁ လုံး ၁ ကော ရင် ရန် ဖြစ်']


In [22]:
#with symspell word segmentaion
#Consonant error, sequence error

sent = "ကြက်ဉ၁လုံး၁၅၀၀ ြပောရျ်ရန်ဖြစ်"

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', corrected_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', distance_sum=17, log_prob_sum=-101.53361432258968)
Edit distance 2
Composition(segmented_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', corrected_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', distance_sum=17, log_prob_sum=-101.53361432258968)
Edit distance 3
Composition(segmented_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', corrected_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', distance_sum=17, log_prob_sum=-101.53361432258968)
Edit distance 4
Composition(segmented_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', corrected_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', distance_sum=17, log_prob_sum=-101.53361432258968)
Edit distance 5
Composition(segmented_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', corrected_string='ကြက် ဉ၁ လုံး ၁ ၅ ၀ ၀ ြပောရျ် ရန် ဖြစ်', distance_sum=17, log_prob_sum=-101.53361432258968)
Edit distance 6
Compositi

In [23]:
# dialect error
sent = "စပ်စု လတ် သာ အေ "

for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(sent,i))

Edit distance 1 ['စပ်စ် လတ် သာ အေ']
Edit distance 2 ['စပ်စ် လတ် သာ အေ']
Edit distance 3 ['စပ်စ် လတ် သာ အေ']
Edit distance 4 ['စပ်စ် လတ် သာ အေ']
Edit distance 5 ['စပ်စ် လတ် သာ အေ']
Edit distance 6 ['စပ်စ် လတ် သာ အေ']


In [24]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(sent,i))

1-gram  ['စုပ် လတ် သာ အေ']
2-gram  ['စု လတ် သာ အေ']
3-gram  ['စပ်စ် လတ် သာ အေ']
4-gram  ['စပ်စ် လတ် သာ အေ']
5-gram  ['စပ်စ် လတ် သာ အေ']
6-gram  ['စပ်စ် လတ် သာ အေ']


In [25]:
#with symspell word segmentaion
# dialect error

sent = "စပ်စုလတ်သာအေ "

for i in range(1,7):
  print("Edit distance {}".format(i))
  symspell_word = SymSpell(max_dictionary_edit_distance=1)
  symspell_word._distance_algorithm = DistanceAlgorithm.DAMERAU_OSA_FAST
  symspell_word.load_bigram_dictionary(bigram_path, 0, 2)
  symspell_word.load_dictionary(dictionary_path, 0, 1)

  sent_seg = symspell_word.word_segmentation(sent, 0)
  print(sent_seg)

Edit distance 1
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)
Edit distance 2
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)
Edit distance 3
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)
Edit distance 4
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)
Edit distance 5
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)
Edit distance 6
Composition(segmented_string='စပ် စု လတ် သာ အေ ', corrected_string='စပ် စု လတ် သာ အေ ', distance_sum=4, log_prob_sum=-57.789514523046)


In [43]:
# Common spelling mistakes Consonant errors
common1 = "ည က အိမ် မက် က ကွာ ၊ "
common2 = "မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စညး ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော"

In [44]:
for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(common1,i))

Edit distance 1 ['ည က အိမ် မက် က ကွာ ၊']
Edit distance 2 ['ည က အိမ် မက် က ကွာ ၊']
Edit distance 3 ['ည က အိမ် မက် က ကွာ ၊']
Edit distance 4 ['ည က အိမ် မက် က ကွာ ၊']
Edit distance 5 ['ည က အိမ် မက် က ကွာ ၊']
Edit distance 6 ['ည က အိမ် မက် က ကွာ ၊']


In [45]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(common1,i))

1-gram  ['ည က အိမ် မက် က ကွာ ၊']
2-gram  ['ည က အိမ် မက် က ကွာ ၊']
3-gram  ['ည က အိမ် မက် က ကွာ ၊']
4-gram  ['ည က အိမ် မက် က ကွာ ၊']
5-gram  ['ည က အိမ် မက် က ကွာ ၊']
6-gram  ['ည က အိမ် မက် က ကွာ ၊']


In [46]:
for i in range(1,7):
  print("Edit distance {}".format(i),mySymSpell(common2,i))

Edit distance 1 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
Edit distance 2 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
Edit distance 3 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
Edit distance 4 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
Edit distance 5 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
Edit distance 6 ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']


In [47]:
for i in range(1,7):
  print("{}-gram ".format(i),ngram_similarity(common2,i))

1-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
2-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
3-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
4-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
5-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
6-gram  ['မက် ရင်း က နေ ရောင် ရမ်း ပြီး ပျောက် သွား တဲ့ ပစ္စည်း ကို အ ပြင် မှာ ပါ လိုက် ရှာ နေ ရော']
